In [1]:
# !wget https://huggingface.co/datasets/Babelscape/rebel-dataset/resolve/main/rebel_dataset.zip
# !unzip rebel_dataset.zip

In [21]:
!ls en_*.jsonl

en_test.jsonl  en_train.jsonl  en_val.jsonl


In [2]:
!head -n 1 en_val.jsonl

{"docid": "30113940", "title": "Transportation Journal", "uri": "Q7835172", "text": "Transportation Journal is an academic journal devoted to transportation, logistics and related fields. The journal is published quarterly by the Penn State University Press on behalf of the American Society of Transportation and Logistics.", "entities": [{"uri": "Q737498", "boundaries": [29, 45], "surfaceform": "academic journal", "annotator": "Me"}, {"uri": "Q7163300", "boundaries": [145, 172], "surfaceform": "Penn State University Press", "annotator": "Me"}, {"uri": "Q296160", "boundaries": [190, 238], "surfaceform": "American Society of Transportation and Logistics", "annotator": "Me"}, {"uri": "Q7835172", "boundaries": [0, 22], "surfaceform": "Transportation Journal", "annotator": "Me"}], "triples": [{"subject": {"uri": "Q7835172", "boundaries": [0, 22], "surfaceform": "Transportation Journal", "annotator": "Me"}, "predicate": {"uri": "P123", "boundaries": null, "surfaceform": "publisher", "annotat

In [5]:
import re

# article = data[1]

def rebel_format(article):
    prev_len = 0
    count = 0
    for text_paragraph in article['text'].split('\n'):
        if len(text_paragraph) == 0:
            continue
        sentences = re.split(r'(?<=[.])\s', text_paragraph)
        text = ''
        for sentence in sentences:
            text += sentence + ' '
            if any([entity['boundaries'][0] < len(text) + prev_len < entity['boundaries'][1] for entity in article['entities']]):
                continue
            entities = sorted([entity for entity in article['entities'] if prev_len < entity['boundaries'][1] <= len(text)+prev_len], key=lambda tup: tup['boundaries'][0])
            decoder_output = '<triplet> '
            for int_ent, entity in enumerate(entities):
                triplets = sorted([triplet for triplet in article['triples'] if triplet['subject'] == entity and prev_len< triplet['subject']['boundaries'][1]<=len(text) + prev_len and prev_len< triplet['object']['boundaries'][1]<=len(text)+ prev_len], key=lambda tup: tup['object']['boundaries'][0])
                if len(triplets) == 0:
                    continue
                decoder_output += entity['surfaceform'] + ' <subj> '
                for triplet in triplets:
                    decoder_output += triplet['object']['surfaceform'] + ' <obj> '  + triplet['predicate']['surfaceform'] + ' <subj> '
                decoder_output = decoder_output[:-len(' <subj> ')]
                decoder_output += ' <triplet> '
            decoder_output = decoder_output[:-len(' <triplet> ')]
            count += 1
            prev_len += len(text)

            if len(decoder_output) == 0:
                text = ''
                continue

            text = re.sub('([\[\].,!?()])', r' \1 ', text.replace('()', ''))
            text = re.sub('\s{2,}', ' ', text)

            a = {
                'title': article['title'],
                'context': text,
                'id': article['uri'] + '-' + str(count),
                'triplets': decoder_output,
            }
            return a

In [7]:
l

{'docid': '30113940',
 'title': 'Transportation Journal',
 'uri': 'Q7835172',
 'text': 'Transportation Journal is an academic journal devoted to transportation, logistics and related fields. The journal is published quarterly by the Penn State University Press on behalf of the American Society of Transportation and Logistics.',
 'entities': [{'uri': 'Q737498',
   'boundaries': [29, 45],
   'surfaceform': 'academic journal',
   'annotator': 'Me'},
  {'uri': 'Q7163300',
   'boundaries': [145, 172],
   'surfaceform': 'Penn State University Press',
   'annotator': 'Me'},
  {'uri': 'Q296160',
   'boundaries': [190, 238],
   'surfaceform': 'American Society of Transportation and Logistics',
   'annotator': 'Me'},
  {'uri': 'Q7835172',
   'boundaries': [0, 22],
   'surfaceform': 'Transportation Journal',
   'annotator': 'Me'}],
 'triples': [{'subject': {'uri': 'Q7835172',
    'boundaries': [0, 22],
    'surfaceform': 'Transportation Journal',
    'annotator': 'Me'},
   'predicate': {'uri': 'P

In [19]:
import json
from tqdm import tqdm

data = []
with open('rebel-val.jsonl', 'w') as fopen:
    with open('en_val.jsonl') as fread:
        for l in tqdm(fread):
            l = json.loads(l)
            a = rebel_format(l)
            if a:
                fopen.write(f'{json.dumps(a)}\n')

152673it [00:06, 22817.44it/s]


In [20]:
!head -n 1 rebel-val.jsonl

{"title": "Chelopech Hill", "context": "Chelopech Hill ( , \u2018Halm Chelopech\u2019 \\'h&lm che-lo-'pech\\ ) is the ice-covered hill rising to 946 m in the north foothills of Detroit Plateau on Trinity Peninsula in Graham Land , Antarctica . ", "id": "Q5090011-1", "triplets": "<triplet> Trinity Peninsula <subj> Graham Land <obj> part of <subj> Antarctica <obj> continent <triplet> Graham Land <subj> Antarctica <obj> continent"}


In [22]:
with open('rebel-test.jsonl', 'w') as fopen:
    with open('en_test.jsonl') as fread:
        for l in tqdm(fread):
            l = json.loads(l)
            a = rebel_format(l)
            if a:
                fopen.write(f'{json.dumps(a)}\n')

152836it [00:06, 22850.83it/s]


In [23]:
with open('rebel-train.jsonl', 'w') as fopen:
    with open('en_train.jsonl') as fread:
        for l in tqdm(fread):
            l = json.loads(l)
            a = rebel_format(l)
            if a:
                fopen.write(f'{json.dumps(a)}\n')

2754388it [01:57, 23425.76it/s]


In [24]:
!split -l 200000 -d --additional-suffix=.splitted rebel-train.jsonl rebel-train

In [26]:
!split -l 200000 -d --additional-suffix=.splitted rebel-test.jsonl rebel-test

In [28]:
!split -l 200000 -d --additional-suffix=.splitted rebel-val.jsonl rebel-val

In [15]:
def parse_rebel(text):
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace('<s>', '').replace("<pad>", '').replace('</s>', '').split():
        if token == '<triplet>':
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(), 'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == '<subj>':
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(), 'tail': object_.strip()})
            object_ = ''
        elif token == '<obj>':
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(), 'tail': object_.strip()})
    return triplets

In [18]:
parse_rebel('<triplet> Trinity Peninsula <subj> Graham Land <obj> part of <subj> Antarctica <obj> continent <triplet> Graham Land <subj> Antarctica <obj> continent')

[{'head': 'Trinity Peninsula', 'type': 'part of', 'tail': 'Graham Land'},
 {'head': 'Trinity Peninsula', 'type': 'continent', 'tail': 'Antarctica'},
 {'head': 'Graham Land', 'type': 'continent', 'tail': 'Antarctica'}]

In [ ]:
triplet subj obj subj obj

In [ ]:
article

In [ ]:
kelm = []
with open('quadruples-train21.splitted.translated') as fopen:
    for l in fopen:
        kelm.append(json.loads(l))
        if len(kelm) == 100:
            break
            
kelm

In [ ]:
filtered_kelm = []
for k in kelm:
    for k_ in k:
        if len(k_['triples']) > 2:
            filtered_kelm.append(k_)
            
len(filtered_kelm)

In [ ]:
filtered_kelm[0]

In [ ]:
triples

In [ ]:
def dict_to_list(triplets):
    q = []
    for no, triple in enumerate(triplets):
        q.append([triple['head'], triple['type'], triple['tail']])
    return q

def rebel_format(triplets):
    """
    Convert
    [['Bruno Santana', 'participant of', '2004 Summer Olympics'],
    ['Bruno Santana', 'participant of', '2008 Summer Olympics'],
    ['Bruno Santana', 'country of citizenship', 'Brazil']]
    to rebel format,
    <triplet> Bruno Santana <subj> 2004 Summer Olympics <obj> participant of <subj> 2008 Summer Olympics <obj> participant of <subj> Brazil <obj> country of citizenship
    """
    q = []
    for no, triple in enumerate(triplets):
        obj = ['<obj>'] + triple[1].split()
        subj = ['<subj>'] + triple[2].split()
        if no > 0 and triple[0] == triplets[no - 1][0]:
            q.extend(subj + obj)
        else:
            triplet = ['<triplet>'] + triple[0].split()
            q.extend(triplet + subj + obj)
    return ' '.join(q)

In [ ]:
triples = filtered_kelm[1]['triples']
q = rebel_format(triples)

In [ ]:
dict_to_list(parse_rebel(q))